In [1]:
# import all the libraries
import supervision as sv
# supervision==0.19.0
import cv2
import numpy as np

In [3]:
# dowmload model
from ultralytics import YOLO
model = YOLO('yolov8s.pt')

100%|███████████████████████████████████████| 21.5M/21.5M [01:04<00:00, 348kB/s]


In [5]:
# connect model to device (mps, cuda or cpu)
device = 'mps'
model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

In [ ]:
# connecting to our firebase database

import firebase_admin

from firebase_admin import credentials
from firebase_admin import db


cred = credentials.Certificate("sirius-6b2b4-firebase-adminsdk-w0nf0-d148ad0356.json")

firebase_admin.initialize_app(cred, {'databaseURL': 'https://sirius-6b2b4-default-rtdb.europe-west1.firebasedatabase.app/'})

ref_time = db.reference("/waitingtime")
ref_count = db.reference("/countofpeople")

In [57]:
# 1st way of calculating the waiting time

# get all variables that we need to work with supevision Detections object (creating polygons, tracker, box and zone annotators)
video_info = sv.VideoInfo.from_video_path('videos/file5.mp4')
byte_track = sv.ByteTrack(frame_rate=video_info.fps)
frames_generator = sv.get_video_frames_generator('videos/file5.mp4', stride=5)
time = 0
polygon_entry = np.array([
[495, 315],[639, 403],[755, 319],[555, 215],[495, 311]
])
polygon_queue = np.array([
[8, 83],[8, 251],[192, 203],[808, 571],[1016, 567],[1028, 295],[244, 87],[100, 127],[101, 63],[7, 83]
])
polygon_table = np.array([
[39, 209],[55, 253],[175, 229],[159, 169],[51, 181],[35, 205]
])
polygon_check = np.array([
[206, 106],[218, 198],[182, 206],[170, 114],[206, 106]
])

box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=0.5)
zone_entry = sv.PolygonZone(polygon=polygon_entry, frame_resolution_wh=video_info.resolution_wh)
zone_queue = sv.PolygonZone(polygon=polygon_queue, frame_resolution_wh=video_info.resolution_wh)
zone_table = sv.PolygonZone(polygon=polygon_table, frame_resolution_wh=video_info.resolution_wh)
zone_check = sv.PolygonZone(polygon=polygon_check, frame_resolution_wh=video_info.resolution_wh)
zone_annotator_queue = sv.PolygonZoneAnnotator(zone=zone_queue, color=sv.Color.white(), thickness=3, text_thickness=3, text_scale=2)
zone_annotator_table = sv.PolygonZoneAnnotator(zone=zone_table, color=sv.Color.white(), thickness=3, text_thickness=3, text_scale=2)
zone_annotator_check = sv.PolygonZoneAnnotator(zone=zone_check, color=sv.Color.white(), thickness=3, text_thickness=3, text_scale=2)


# initialize arrays that we will use for saving tracked ids of people walking into the zone and counting the time at what they did it
entry_ids = []
last_entry_time = 0
times = []
start = time.time()

# now iterate through frames and saving everything we need
with sv.VideoSink(target_path='videos_results/file5_result.mp4', video_info=video_info) as sink:
  for frame in frames_generator:
      time += 5/10
      results = model(frame, imgsz=1280)[0]
      detections = sv.Detections.from_ultralytics(results)
      detections = detections[detections.class_id==0]
      detections = byte_track.update_with_detections(detections=detections)
      zone_entry.trigger(detections=detections)
      CONST = 7 if (zone_check.trigger(detections=detections)==True).sum() > 0 else 0
      count = (zone_queue.trigger(detections=detections)==True).sum() - (zone_table.trigger(detections=detections)==True).sum() + CONST
      if start >= 10:
          ref_count.set(int(count)) # change our firebase database value for number of people in the queue
      detections = detections[zone_entry.trigger(detections=detections)==True]
      print(detections.tracker_id)
      for id in detections.tracker_id:
          if id not in entry_ids:
              if len(entry_ids) == 0:
                  last_entry_time = time
                  times.append((time - last_entry_time) * count)
                  last_entry_time = time
                  entry_ids.append(id)
              else:
                  times.append((time - last_entry_time) * count)
                  last_entry_time = time
                  entry_ids.append(id)
      
      if time % 10 == 0 and len(times) > 2:
          ref_time.set(int(sum(times[-1:])/len(times[-1:]))) # changing database value for waiting time

      box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=0.5)
      labels = [
          f'#{tracker_id}'
          for tracker_id
          in detections.tracker_id]
      frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
      frame = zone_annotator_check.annotate(zone_annotator_queue.annotate(zone_annotator_table.annotate(scene=frame)))

      sink.write_frame(frame=frame)

print(times)
print(f'average time for waiting: {sum(times)/len(times)}s')

SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: white is deprecated: `Color.white()` is deprecated and will be removed in `supervision-0.22.0`. Use `Color.WHITE` instead.
SupervisionWarnings: white is deprecated: `Color.white()` is deprecated and will be removed in `supervision-0.22.0`. Use `Color.WHITE` instead.
SupervisionWarnings: white is deprecated: `Color.white()` is deprecated and will be removed in `supervision-0.22.0`. Use `Color.WHITE` instead.


0: 736x1280 30 persons, 2 backpacks, 1 handbag, 1 cup, 4 chairs, 1 potted plant, 2 dining tables, 1 tv, 1 cell phone, 1 refrigerator, 140.0ms
Speed: 274.4ms preprocess, 140.0ms inference, 88.1ms postprocess per image at shape (1, 3, 736, 1280)
[]

0: 736x1280 32 persons, 1 backpack, 1 cup, 4 chairs, 1 potted plant, 3 dining tables, 1 cell phone, 1 refrigerator, 33.4ms
Speed: 5.1ms preprocess, 33.4ms inference, 57.6ms postprocess per image at shape (1, 3, 736, 1280)
[]

0: 736x1280 29 persons, 1 cup, 4 chairs, 1 potted plant, 2 dining tables, 1 tv, 1 cell phone, 1 refrigerator, 27.5ms
Speed: 4.5ms preprocess, 27.5ms inference, 14.8ms postprocess per image at shape (1, 3, 736, 1280)
[]



SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


0: 736x1280 29 persons, 1 handbag, 1 cup, 4 chairs, 1 potted plant, 2 dining tables, 1 cell phone, 1 refrigerator, 25.5ms
Speed: 5.0ms preprocess, 25.5ms inference, 12.6ms postprocess per image at shape (1, 3, 736, 1280)
[]

0: 736x1280 28 persons, 3 backpacks, 1 tie, 1 cup, 4 chairs, 1 potted plant, 2 dining tables, 1 cell phone, 1 refrigerator, 26.1ms
Speed: 4.6ms preprocess, 26.1ms inference, 13.0ms postprocess per image at shape (1, 3, 736, 1280)
[]

0: 736x1280 27 persons, 1 backpack, 1 cup, 6 chairs, 1 potted plant, 1 dining table, 2 cell phones, 1 refrigerator, 25.4ms
Speed: 4.6ms preprocess, 25.4ms inference, 12.4ms postprocess per image at shape (1, 3, 736, 1280)
[]



SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


...

SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead



0: 736x1280 27 persons, 6 backpacks, 2 handbags, 1 suitcase, 1 cup, 3 chairs, 1 potted plant, 1 dining table, 1 cell phone, 1 refrigerator, 33.2ms
Speed: 18.7ms preprocess, 33.2ms inference, 59.6ms postprocess per image at shape (1, 3, 736, 1280)
[45053 44871]

0: 736x1280 30 persons, 6 backpacks, 3 handbags, 1 suitcase, 1 cup, 3 chairs, 1 dining table, 1 tv, 1 cell phone, 1 refrigerator, 28.4ms
Speed: 4.4ms preprocess, 28.4ms inference, 53.8ms postprocess per image at shape (1, 3, 736, 1280)
[45053 44871]

0: 736x1280 32 persons, 5 backpacks, 1 handbag, 1 cup, 2 chairs, 1 potted plant, 1 dining table, 1 cell phone, 1 refrigerator, 27.1ms
Speed: 4.4ms preprocess, 27.1ms inference, 13.3ms postprocess per image at shape (1, 3, 736, 1280)
[44927 45053 44871]



SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


0: 736x1280 32 persons, 6 backpacks, 1 tie, 1 cup, 3 chairs, 1 potted plant, 1 dining table, 1 cell phone, 1 refrigerator, 25.2ms
Speed: 4.5ms preprocess, 25.2ms inference, 56.0ms postprocess per image at shape (1, 3, 736, 1280)
[44927 45053 44871]

0: 736x1280 32 persons, 4 backpacks, 1 tie, 1 cup, 3 chairs, 1 potted plant, 1 dining table, 1 cell phone, 1 refrigerator, 30.4ms
Speed: 4.4ms preprocess, 30.4ms inference, 11.5ms postprocess per image at shape (1, 3, 736, 1280)
[44927 45053 44871]

0: 736x1280 32 persons, 3 backpacks, 1 handbag, 1 tie, 1 cup, 3 chairs, 1 potted plant, 1 dining table, 1 cell phone, 1 refrigerator, 25.5ms
Speed: 4.8ms preprocess, 25.5ms inference, 52.9ms postprocess per image at shape (1, 3, 736, 1280)
[44927 45053 44871]
[0.0, 57.5, 276.0, 108.0, 126.0, 69.0, 110.0, 299.0, 195.0]
average time for waiting: 137.83333333333334s


SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


In [25]:
# 2 way of calculating waitiing time

# do everything the same except for other polygons
video_info = sv.VideoInfo.from_video_path('videos/queues_short.mp4')
byte_track = sv.ByteTrack(frame_rate=video_info.fps)
frames_generator = sv.get_video_frames_generator('videos/queues_short.mp4')
time = 0
polygon_left = np.array([
[11, 583],[19, 699],[171, 703],[159, 575],[15, 579]
])
polygon_right = np.array([
[964, 541],[1068, 489],[972, 393],[1068, 345],[1152, 405],[1132, 705],[980, 709],[964, 537]
])

box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=0.5)
zone_left = sv.PolygonZone(polygon=polygon_left, frame_resolution_wh=video_info.resolution_wh)
zone_right = sv.PolygonZone(polygon=polygon_right, frame_resolution_wh=video_info.resolution_wh)
zone_annotator_left = sv.PolygonZoneAnnotator(zone=zone_left, color=sv.Color.white(), thickness=3, text_thickness=3, text_scale=2)
zone_annotator_right = sv.PolygonZoneAnnotator(zone=zone_right, color=sv.Color.white(), thickness=3, text_thickness=3, text_scale=2)


# here we initialize twice more arrays (as we want to see the number of people enetring and exiting the queue)
entry_ids = []
exit_ids = []
entry_times = []
exit_times = []
with sv.VideoSink(target_path='videos_results/queues_short_results.mp4', video_info=video_info) as sink:
  for frame in frames_generator:
      time += 1/video_info.fps
      results = model(frame, imgsz=1280)[0]
      detections = sv.Detections.from_ultralytics(results)
      detections = detections[detections.class_id==0]
      zone_left.trigger(detections=detections)
      zone_right.trigger(detections=detections)
      detections = byte_track.update_with_detections(detections=detections)

      for id in detections.tracker_id[zone_left.trigger(detections=detections)==True]:
          if id not in exit_ids:
              exit_times.append(time)
              exit_ids.append(id)
      for id in detections.tracker_id[zone_right.trigger(detections=detections)==True]:
          if id not in entry_ids:
              entry_times.append(time)
              entry_ids.append(id)

      box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=0.5)

      labels_left = [
          f'#{tracker_id}'
          for tracker_id
          in detections.tracker_id[zone_left.trigger(detections=detections)==True]]
      labels_right = [
          f'#{tracker_id}'
          for tracker_id
          in detections.tracker_id[zone_right.trigger(detections=detections)==True]]

      frame = box_annotator.annotate(scene=frame, detections=detections[zone_left.trigger(detections=detections)==True], labels=labels_left)
      frame = box_annotator.annotate(scene=frame, detections=detections[zone_right.trigger(detections=detections)==True], labels=labels_right)
      frame = zone_annotator_left.annotate(zone_annotator_right.annotate(scene=frame))

      sink.write_frame(frame=frame)

# Here we won't update the database values for waiting time as here we need to have videos of the empty crowd so we can set the time of
# the first person entering and exiting the queue, however we have the final arrays as an example
print(entry_times)
print(exit_times)


0: 736x1280 33 persons, 1 bottle, 3 cups, 2 bowls, 13 chairs, 3 dining tables, 1 refrigerator, 44.1ms
Speed: 22.8ms preprocess, 44.1ms inference, 49.8ms postprocess per image at shape (1, 3, 736, 1280)
[]
[]
Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=array([], dtype=int64), data={})
Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=array([], dtype=int64), data={})



SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: white is deprecated: `Color.white()` is deprecated and will be removed in `supervision-0.22.0`. Use `Color.WHITE` instead.
SupervisionWarnings: white is deprecated: `Color.white()` is deprecated and will be removed in `supervision-0.22.0`. Use `Color.WHITE` instead.
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


0: 736x1280 29 persons, 1 bottle, 3 cups, 2 bowls, 13 chairs, 4 dining tables, 1 refrigerator, 27.8ms
Speed: 2.5ms preprocess, 27.8ms inference, 52.5ms postprocess per image at shape (1, 3, 736, 1280)
[]
[]
Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=array([], dtype=int64), data={})
Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=array([], dtype=int64), data={})

0: 736x1280 30 persons, 1 bottle, 2 cups, 2 bowls, 13 chairs, 3 dining tables, 1 refrigerator, 29.9ms
Speed: 2.7ms preprocess, 29.9ms inference, 13.1ms postprocess per image at shape (1, 3, 736, 1280)
[]
[]
Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=array([], dtype=int64), data={})
Detections(xyxy=array([], shape=(0, 4), dt

SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


0: 736x1280 28 persons, 1 bottle, 1 cup, 2 bowls, 12 chairs, 3 dining tables, 1 refrigerator, 25.3ms
Speed: 2.8ms preprocess, 25.3ms inference, 54.2ms postprocess per image at shape (1, 3, 736, 1280)
[]
[]
Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=array([], dtype=int64), data={})
Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=array([], dtype=int64), data={})

0: 736x1280 28 persons, 1 bottle, 2 bowls, 12 chairs, 3 dining tables, 1 refrigerator, 30.0ms
Speed: 2.9ms preprocess, 30.0ms inference, 51.9ms postprocess per image at shape (1, 3, 736, 1280)
[]
[]
Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=array([], dtype=int64), data={})
Detections(xyxy=array([], shape=(0, 4), dtype=float

...

0: 736x1280 25 persons, 1 bottle, 1 cup, 2 bowls, 8 chairs, 5 dining tables, 1 laptop, 25.3ms
Speed: 2.7ms preprocess, 25.3ms inference, 12.7ms postprocess per image at shape (1, 3, 736, 1280)
['#44169']
[]
Detections(xyxy=array([[    -3.5103,      347.73,      109.21,      618.17]], dtype=float32), mask=None, confidence=array([    0.93291], dtype=float32), class_id=array([0]), tracker_id=array([44169]), data={})
Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=array([], dtype=int64), data={})

0: 736x1280 24 persons, 1 bottle, 1 cup, 2 bowls, 9 chairs, 5 dining tables, 1 laptop, 25.7ms
Speed: 2.7ms preprocess, 25.7ms inference, 13.2ms postprocess per image at shape (1, 3, 736, 1280)
['#44169']
[]
Detections(xyxy=array([[    -3.0965,      348.97,      108.41,      617.99]], dtype=float32), mask=None, confidence=array([    0.93616], dtype=float32), class_id=array([0]), tracker_id=array([44

SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


0: 736x1280 25 persons, 1 bottle, 1 cup, 2 bowls, 11 chairs, 5 dining tables, 26.6ms
Speed: 2.6ms preprocess, 26.6ms inference, 51.9ms postprocess per image at shape (1, 3, 736, 1280)
['#44169']
[]
Detections(xyxy=array([[    -2.9941,      351.69,      106.32,      618.27]], dtype=float32), mask=None, confidence=array([    0.93493], dtype=float32), class_id=array([0]), tracker_id=array([44169]), data={})
Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=array([], dtype=int64), data={})

0: 736x1280 23 persons, 1 bottle, 1 cup, 2 bowls, 9 chairs, 5 dining tables, 28.4ms
Speed: 2.7ms preprocess, 28.4ms inference, 48.7ms postprocess per image at shape (1, 3, 736, 1280)
['#44169']
[]
Detections(xyxy=array([[    -2.9085,      352.17,      105.64,      618.32]], dtype=float32), mask=None, confidence=array([    0.93495], dtype=float32), class_id=array([0]), tracker_id=array([44169]), data={})
Det

SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


[1.400000000000001, 4.899999999999992, 16.133333333333294, 16.799999999999994, 17.600000000000033, 17.76666666666671, 18.066666666666723, 18.633333333333418, 19.933333333333483, 20.400000000000173, 22.800000000000292]
[43866, 43908, 44018, 44020, 44022, 44028, 44033, 44029, 44059, 44062, 44085]
[0.49999999999999994, 2.8666666666666663, 6.9333333333333185, 16.69999999999999, 18.500000000000078, 20.633333333333518, 23.200000000000312, 31.53333333333406, 32.96666666666736]
[43835, 43863, 43899, 43980, 43942, 44051, 44045, 44064, 44169]


In [44]:
import time

In [50]:
# this is the same code which updates the information about how many people are at the tables so we can calcu

video_info = sv.VideoInfo.from_video_path('videos/table.mp4')
polygon = np.array([
[554, 697],[62, 201],[62, 145],[474, 33],[1210, 237],[1210, 689],[558, 693]
])
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=0.5)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=3, text_thickness=3, text_scale=2)

with sv.VideoSink(target_path='/Users/vladimirkalajcidi/Downloads/table_result', video_info=video_info) as sink:

    for frame in sv.get_video_frames_generator('videos/table.mp4', stride=300):
        time.sleep(10)

        results = model(frame, imgsz=1280)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[detections.class_id == 0]
        zone.trigger(detections=detections)
        print(f'People at the tables {(zone.trigger(detections=detections)==True).sum()}')
        ref.set(int((zone.trigger(detections=detections)==True).sum()))

        box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=0.5)
        labels = [f"{model.names[class_id]} {confidence:0.2f}" for confidence, class_id in \
                  zip(detections.confidence, detections.class_id)]
        frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
        frame = zone_annotator.annotate(scene=frame)

        sink.write_frame(frame=frame)

SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: white is deprecated: `Color.white()` is deprecated and will be removed in `supervision-0.22.0`. Use `Color.WHITE` instead.
[ERROR:0@69260.886] global cap.cpp:645 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:430: error: (-215:Assertion failed) !filename_pattern.empty() in function 'open'





0: 736x1280 22 persons, 2 cups, 38 chairs, 4 dining tables, 1 cell phone, 122.1ms
Speed: 189.5ms preprocess, 122.1ms inference, 11.1ms postprocess per image at shape (1, 3, 736, 1280)
People at the tables 18


SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead



0: 736x1280 20 persons, 3 cups, 30 chairs, 2 dining tables, 2 laptops, 1 cell phone, 47.0ms
Speed: 66.4ms preprocess, 47.0ms inference, 15.1ms postprocess per image at shape (1, 3, 736, 1280)
People at the tables 19


SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


...

SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead



0: 736x1280 23 persons, 3 cups, 1 bowl, 32 chairs, 2 dining tables, 87.4ms
Speed: 156.7ms preprocess, 87.4ms inference, 65.1ms postprocess per image at shape (1, 3, 736, 1280)
People at the tables 20


SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead



0: 736x1280 21 persons, 1 wine glass, 2 cups, 1 bowl, 33 chairs, 3 dining tables, 1 cell phone, 41.9ms
Speed: 86.6ms preprocess, 41.9ms inference, 304.5ms postprocess per image at shape (1, 3, 736, 1280)
People at the tables 20


SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
